In [1]:
%matplotlib inline

In [2]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import tensorflow as tf

In [4]:
import pandas as pd
from scipy import signal
from scipy.io import wavfile
import seaborn as sns
import librosa
import librosa.display
from sklearn.preprocessing import StandardScaler

In [5]:
import os
import sys
import re

In [16]:
import time
from datetime import datetime, timedelta

In [6]:
# Tensorflow setup
sess = None
tf.logging.set_verbosity(tf.logging.INFO)

def reset_vars():
    """Initializes all tf variables"""
    sess.run(tf.global_variables_initializer())

def reset_tf():
    """Closes the current tf session and opens new session"""
    global sess
    if sess:
        sess.close()
    tf.reset_default_graph()
    sess = tf.Session()

In [7]:
# Function to initialize weights with small positive numbers
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# Function to initialize biases with constant positive number
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
LABELS_REQUIRED = ['yes', 'no', 'up', 'down', 'left',
                   'right', 'on', 'off', 'stop', 'go',
                   'silence']

In [9]:
num2label = {i+1:l for i, l in enumerate(LABELS_REQUIRED)}
num2label[0] = 'unknown'
label2num = {v:k for k, v in num2label.items()}

In [10]:
reset_tf()

# Model parameters
melspec_shape = (64, 126)  # Shape of Mel spectrum data
melspec_size = 64 * 126
mfcc_shape = (19, 126)     # Shape of MFCC data
mfcc_size = 19 * 126
sf_size = 122              # Length of 1D feature arrays e.g. ZCR and RMSE

n_classes = len(label2num)

batch_size = 50
num_iterations = 5000
display_step = 100

learning_rate = 2e-4
dropout_prob = 0.60         # Dropout, probability to keep units

In [11]:
print "Total feature size:  {}".format(melspec_size + mfcc_size + sf_size)

Total feature size:  10580


In [18]:
# Placeholder variables input
x_mel = tf.placeholder(tf.float32, [None, melspec_size], name='x_mel')
x_mfcc = tf.placeholder(tf.float32, [None, mfcc_size], name='x_mfcc')
x_zcr = tf.placeholder(tf.float32, [None, sf_size], name='x_zcr')
x_rmse = tf.placeholder(tf.float32, [None, sf_size], name='x_rsme')

In [19]:
# Placeholder variables output
y_true = tf.placeholder(tf.float32, shape=[None, n_classes], name='y_true')
y_true_class = tf.argmax(y_true, 1)

In [14]:
dropout = tf.placeholder(tf.float32, name='dropout')

In [83]:
# Prediction from model
y_pred = conv_net_speech_model(x_mel, x_mfcc, x_zcr, x_rmse)

In [84]:
# Cross entropy loss function with softmax then takes mean
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
tf.summary.scalar('loss', loss)

# Train and backprop gradients function
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Evaluation and accuracy
y_pred_class = tf.argmax(y_pred, 1)
correct_prediction = tf.equal(y_pred_class, y_true_class)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [85]:
# MODEL
def conv_net_speech_model(x_mel, x_mfcc, x_zcr, x_rmse, w=None, b=None, d=None):
    
    # TODO: Build model here!

    
    
    
    
    
    return tf.placeholder(tf.float32, shape=[None, n_classes])

In [25]:
# TRAINING
def run_optimize(num_iterations, min_loss=0):
    # Start-time
    start_time = time.time()

    for i in xrange(num_iterations):
        
        # TODO: Build training step here!
        
        
        
        
        if loss_train < min_loss:
            break
    
    # End-time
    end_time = time.time()
    print "Time usage: {}".format(timedelta(seconds=int(round(end_time - start_time))))    
    return

In [20]:
# REFERENCE SIMPLE CONVNET MODEL

# Conv and max pooling layers wrapper functions for simplicity
def conv2d(x, W, s=1):
    return tf.nn.conv2d(x, W, strides=[1, s, s, 1], padding='SAME')


def max_pool_2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


# MODEL
def conv_net_model(x, weights, biases, dropout):
    # Reshape input (note -1 is used to infer shape)
    #   [image number, px width, px height, color channel]
    x_image = tf.reshape(x, [-1, 28, 28, 1])  # image is now 28x28

    # Layer 1:
    conv1 = tf.nn.relu(conv2d(x_image, weights['wc1']) + biases['bc1'])
    pool1 = max_pool_2d(conv1)                # image is now 14x14

    # Layer 2:
    conv2 = tf.nn.relu(conv2d(pool1, weights['wc2']) + biases['bc2'])
    pool2 = max_pool_2d(conv2)                # image is now 7x7

    # Flatten to fit fc layer
    #   [image number, 7*7*64 inputs]
    pool2_flat = tf.reshape(pool2, [-1, 3136])
    
    # Layer 3:
    fc1 = tf.nn.relu(tf.matmul(pool2_flat, weights['wd1']) + biases['bd1'])

    # Dropout
    fc1_drop = tf.nn.dropout(fc1, dropout)

    # Readout layer to classify into 10 catagories
    y = tf.matmul(fc1_drop, weights['out']) + biases['out']

    return y